In [ ]:
import numpy as np
import random

In [ ]:
probRECOMBINACAO = 0.9
probMUTACAO = 0.4
tamPOPULACAO = 100
contFitness = 0 #contador para ver quando chega em 10 mil

In [ ]:
# Gerando uma população
new_population = [np.random.permutation(np.arange(1, 9)) for _ in range(tamPOPULACAO)] # Inicializando N indivíduos formados por permutações aleatórias de 1 a 8

print(new_population) # Mostrando os indivíduos gerados

[array([6, 7, 8, 2, 4, 3, 5, 1]), array([6, 1, 3, 4, 7, 5, 2, 8]), array([6, 7, 1, 2, 8, 4, 5, 3]), array([2, 4, 7, 1, 6, 8, 3, 5]), array([8, 2, 3, 6, 5, 4, 7, 1]), array([8, 5, 2, 7, 4, 6, 3, 1]), array([1, 2, 3, 5, 4, 7, 8, 6]), array([4, 3, 8, 2, 6, 5, 1, 7]), array([7, 3, 5, 1, 4, 2, 6, 8]), array([1, 5, 2, 8, 4, 6, 7, 3]), array([7, 1, 4, 6, 2, 8, 5, 3]), array([6, 3, 4, 8, 1, 7, 5, 2]), array([1, 2, 4, 5, 6, 7, 8, 3]), array([8, 6, 2, 1, 5, 4, 3, 7]), array([1, 4, 2, 3, 6, 7, 5, 8]), array([5, 6, 1, 4, 8, 2, 7, 3]), array([5, 4, 8, 2, 1, 3, 6, 7]), array([2, 3, 5, 6, 4, 1, 7, 8]), array([3, 6, 4, 5, 7, 8, 2, 1]), array([8, 2, 3, 1, 6, 7, 4, 5]), array([4, 3, 2, 8, 1, 7, 5, 6]), array([6, 7, 4, 5, 2, 3, 8, 1]), array([5, 1, 3, 4, 8, 6, 7, 2]), array([5, 3, 1, 8, 4, 2, 6, 7]), array([4, 2, 8, 1, 7, 5, 6, 3]), array([3, 6, 7, 5, 1, 4, 8, 2]), array([6, 1, 7, 4, 5, 2, 8, 3]), array([8, 3, 4, 5, 6, 2, 7, 1]), array([1, 4, 7, 6, 2, 5, 8, 3]), array([6, 7, 8, 4, 2, 3, 1, 5]), array([7,

In [ ]:
# Função de Fitness

def fitness (vetorRainhas):
  numColisao = 0 # inicialmente vetor colisoao zerado
  global contFitness
  contFitness += 1 #contador para ver quando chega em 10 mil

  for j in range(8):
    for k in range(j+1, 8):
      if abs(j - k) == abs(vetorRainhas[j] - vetorRainhas[k]):
        numColisao += 1
      #precisa criar o que verifica a horizontal por causa das mutacoes
      if vetorRainhas[j] == vetorRainhas[k]:
        numColisao += 2

  return numColisao

# Associando cada indivíduo ao seu fitness
#population = [(individuo, 1/(1+ fitness(individuo))) for individuo in new_population]
population = [(individuo, fitness(individuo)) for individuo in new_population]

In [ ]:
def selecaoPais(populacao): #populacao vai ser uma tupla (ind,fitness)
  todos = populacao.copy()
  random.shuffle(todos) #embaralhando tuplas
  selecionados = []
  divisoes = [todos[i*5 : (i+1)*5] for i in range(20)] # 20 divisoes de 5

  for j in range (20):
   ordenado = sorted(divisoes[j], key=lambda x: x[1])
   selecionados.extend(ordenado[:2]) #selecionando os 2 menores

  return selecionados

In [ ]:
# Fazendo os pares dos pais
def criacaoPares (populacao):
      return [(populacao[i], populacao[i+1]) for i in range(0, len(populacao), 2)]

In [ ]:
# Combinando os indíviudos
# Código para a recombinação

def recombinacao (pai, mae):
  #pegando so o gene
  genPai, _ = pai
  genMae, _ = mae
  permPai = genPai.copy()
  permMae = genMae.copy()
  divisao = random.randint(1, 7) #pos da divisao

  def laco(pai, mae, divisao):
    j=0
    i= divisao + 1
    vet = np.full(8-(divisao),0)
    while True:
      element = mae[i%8]
      if element not in pai[:divisao]:
        vet[j] = element
        j+=1
      i+=1
      if j == 8-divisao:
        break
    return vet

  if random.random() < probRECOMBINACAO:
    filho1 = np.concatenate((permPai[:divisao], laco(permPai,permMae,divisao)))
    filho2 = np.concatenate((permMae[:divisao], laco(permMae,permPai,divisao)))

  else:
    filho1 = permPai
    filho2 = permMae

  return filho1, filho2

In [ ]:
# Código da Mutação genética
def mutacao(individuo):
  individuoMutado = individuo.copy()

  if random.random() < probMUTACAO: # prob de 40% de se mudar
    i, j = random.sample(range(len(individuoMutado)), 2)
    individuoMutado[i], individuoMutado[j] = individuoMutado[j], individuoMutado[i]

  return individuoMutado

In [ ]:
def gerandoFilhos (populacao):
  pares = criacaoPares(populacao)
  filhos = []

  for pai, mae in pares:
    filho1, filho2 = recombinacao(pai, mae) #gera sem mutacao
    mutacaoFilho1 = mutacao(filho1) #aplica a mutacao
    mutacaoFilho2 = mutacao(filho2)
    filhos.extend([mutacaoFilho1, mutacaoFilho2]) #cria um vet de filhos

  return filhos

In [ ]:
def selecaoPop (filhos,pai):
  # o vet de pais tem (ge,fit)
  # o vet de filhos tem (gen)
  genPai = [gen for gen, fit in pais]
  vetPais = genPai.copy()
  vetFilhos = filhos.copy()

  vetTodos = vetPais + vetFilhos
  tuplaTodos = [(ind,(fitness(ind))) for ind in vetTodos]
  ordTupla = sorted(tuplaTodos, key=lambda x: x[1]) #ordenando do menor para o maior

  if (len(vetTodos) > tamPOPULACAO):
    selecionados = ordTupla[:tamPOPULACAO] #selecionando os 100 primeiros (melhores)

  else:
    selecionados = ordTupla

  return selecionados


In [ ]:
# Verificar se a população gerada tem algm com fitness = 0 (combinação perfeita)
def achouRainha (populacao):

  for gen, fit in populacao:
    if fit == 0 :
      print("Achou a combinação:", gen)
      return True
    return False



In [ ]:
def main():
  resultado = False
  contGeracoes=0
  new_population = [np.random.permutation(np.arange(1, 9)) for _ in range(tamPOPULACAO)] # Inicializando N indivíduos formados por permutações aleatórias de 1 a 8
  population = [(individuo, fitness(individuo)) for individuo in new_population]

  while (resultado == False or contFitness<10000):
    contGeracoes += 1
    print("Geração  ", contGeracoes)
    #print(popAtualizada)
    pais = selecaoPais(population)
    #print("Geração de pais: ", pais)
    filhos = gerandoFilhos(pais)
    #print("Geração de filhos: ", filhos)
    population = selecaoPop(filhos, pais)
    resultado = achouRainha(population)

    if resultado == True:
      break


if __name__ == "__main__":
    main()


Geração   1
Geração   2
Geração   3
Geração   4
Geração   5
Geração   6
Geração   7
Geração   8
Geração   9
Geração   10
Geração   11
Geração   12
Geração   13
Geração   14
Geração   15
Geração   16
Geração   17
Geração   18
Geração   19
Geração   20
Geração   21
Geração   22
Geração   23
Geração   24
Geração   25
Geração   26
Geração   27
Geração   28
Geração   29
Geração   30
Geração   31
Geração   32
Geração   33
Geração   34
Geração   35
Geração   36
Geração   37
Geração   38
Geração   39
Geração   40
Geração   41
Achou a combinação: [6 1 5 2 8 3 7 4]
